# Generative text model

In this notebook, we will load and use a generative language model that can produce a continuation for a given text. Learn more about the Text Generation task <a href="https://huggingface.co/tasks/text-generation" target="_blank" rel="noopener">here</a>. Please also refer to the Transformer Extension <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a> to find more information about the UDF used in this notebook.

To execute queries and load data from Exasol database we will be using the <a href="https://github.com/exasol/pyexasol" target="_blank" rel="noopener">`pyexasol`</a> module.

## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the AI-Lab](../main_config.ipynb).
2. [Initialize the Transformer Extension](te_init.ipynb).

## Setup

### Open Secure Configuration Storage

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

## Get language model

To demonstrate the text generation task we will use [Open Pretrained Transformers (OPT)](https://huggingface.co/facebook/opt-125m), a decoder-only pre-trained transformer from Facebook.

We need to load the model from the Huggingface hub into the [BucketFS](https://docs.exasol.com/db/latest/database_concepts/bucketfs/bucketfs.htm). This could potentially be a long process. Unfortunately, we cannot tell exactly when it has finished. The notebook's hourglass may not be a reliable indicator. BucketFS will still be doing some work when the call issued by the notebook returns. Please wait for a few moments after that, before querying the model.

You might see a warning that some weights are newly initialized and the model should be trained on a down-stream task. Please ignore this warning. For the purpose of this demonstration, it is not important, the model should still be able to produce some meaningful output.

In [ ]:
# This is the name of the model at the Huggingface Hub
MODEL_NAME = 'facebook/opt-125m'

In [ ]:
%run utils/model_retrieval.ipynb
load_huggingface_model(ai_lab_config, MODEL_NAME)

## Use language model

Let's put the start of our conversation in a variable.

In [ ]:
MY_TEXT = 'The bar-headed goose can fly at much'

# Make sure our texts can be used in an SQL statement.
MY_TEXT = MY_TEXT.replace("'", "''")

In [ ]:
# Let's put a limit on the length of text the model can generate in one call.
# The limit is specified in the number of characters.
MAX_LENGTH = 30

In [ ]:
from exasol.nb_connector.connections import open_pyexasol_connection
from exasol.nb_connector.language_container_activation import get_activation_sql

# We will be creating a new database session every time we call the model.
# We will have to activate the language container for each of these sessions.
# Here we will get the activation SQL command.
activation_sql = get_activation_sql(ai_lab_config)

At the start, the `MY_TEXT` variable has an initial context. We will update and print this variable at every call to the model.
Please run the next cell multiple times to see how the text evolves.

In [ ]:
sql = f"""
SELECT {ai_lab_config.db_schema}.TE_TEXT_GENERATION_UDF(
    NULL,
    '{ai_lab_config.te_bfs_connection}',
    '{ai_lab_config.te_hf_connection}',
    '{ai_lab_config.te_models_bfs_dir}',
    '{MODEL_NAME}',
    '{MY_TEXT}',
    {MAX_LENGTH},
    True
)
"""

with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    conn.execute(query=activation_sql)
    result = conn.export_to_pandas(query_or_table=sql).squeeze()
    MY_TEXT = result['GENERATED_TEXT']
    # The error can be observed at result['ERROR_MESSAGE']

print(MY_TEXT)
MY_TEXT = MY_TEXT.replace("'", "''")